In [1]:
import pandas as pd
import regex as re
import string
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.linear_model import LinearRegression
from bs4 import BeautifulSoup
import requests


In [4]:
df_true=pd.read_csv('./data/True.csv')
df_true['class']=1
# df_true.head()
df_fake=pd.read_csv('./data/Fake.csv')
df_fake['class']=0
frames=[df_fake,df_true]
df=pd.concat(frames,axis=0)
df.head()

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [5]:
df.columns
df=df.drop(["title","subject","date"],axis=1)

In [7]:
df.head()

,text,class
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0


In [8]:
df=df.sample(frac=1)
df.head()

,text,class
11806,DUBAI (Reuters) - Saudi Arabia s King Salman r...,1
282,"Former Red Sox pitcher Curt Schilling, the hos...",0
18806,"Yesterday, Antifa (anarchists and free-speech ...",0
11669,Singer Joy Villa s album sales spiked signific...,0
4297,WASHINGTON (Reuters) - President Donald Trump ...,1


In [9]:
df.reset_index(inplace = True)
df.drop(["index"], axis = 1, inplace = True)

In [10]:
def clean(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    # text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    # text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text


In [11]:
df["text"]=df["text"].apply(clean)

In [12]:
df["text"]

0        dubai  reuters    saudi arabia s king salman r...
1        former red sox pitcher curt schilling  the hos...
2        yesterday  antifa  anarchists and free speech ...
3        singer joy villa s album sales spiked signific...
4        washington  reuters    president donald trump ...
                               ...                        
44893    donald trump s mouth gets him into trouble a l...
44894    brussels  reuters    the russian authorities  ...
44895    last night  at a rally in melbourne  florida  ...
44896    hysterical with all the evidence available to ...
44897    beirut  reuters    syrian president bashar al ...
Name: text, Length: 44898, dtype: object

In [14]:
X=df["text"]
y=df["class"]
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42,test_size=0.25)


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(X_train)
xv_test = vectorization.transform(X_test)

In [16]:
from sklearn.svm import LinearSVC
svclf=LinearSVC(dual=0,random_state=42)
svclf.fit(xv_train,y_train)

LinearSVC(dual=0, random_state=42)

In [20]:
svclf.score(xv_test,y_test)

0.9956347438752784

In [28]:
dfs=pd.read_csv('./data/sarcasm.csv')
dfs=dfs[['headline','is_sarcastic']]

In [29]:
dfs.head(1)

,headline,is_sarcastic
0,thirtysomething scientists unveil doomsday clo...,1


In [30]:
dfs['headline']=dfs['headline'].apply(clean)

In [31]:
vectorize = TfidfVectorizer()
xs=dfs["headline"]
ys=dfs["is_sarcastic"]
Xs=vectorize.fit_transform(xs)
Xs_train,Xs_test,ys_train,ys_test=train_test_split(Xs,ys,random_state=42,test_size=0.25)

In [32]:
from sklearn.svm import LinearSVC
sarcsvc=LinearSVC()

sarcsvc.fit(Xs_train,ys_train)
def WebScrape(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser') 
    listOfParas=[]
    for i in soup.find_all('p'):
        listOfParas.append(i.text.strip())
    listOfSentences=[]
    for i in listOfParas:
        for sentence in i.split('.'):
            if sentence == "":
                continue
            listOfSentences.append(sentence)
    return listOfSentences

In [35]:
url=str(input())
def CalcSarcasmPercentage(url):
    los=WebScrape(url)
    res=[]
    for x in los:
        new=vectorize.transform([x]).toarray()
        ans=sarcsvc.predict(new)
        res.append(ans[0])
    count=0
    for y in res:
        if y==1:
            count+=1
    percsarc=(count/len(res))*100
    print(percsarc)

In [36]:
CalcSarcasmPercentage(url)

33.33333333333333
